In [9]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
 

In [10]:
 
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether annual income of an individual exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Tue Sep 24 2024', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': "Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the fol

In [11]:
len(X)

48842

In [12]:
len(y)

48842

In [13]:
total=pd.concat([X, y], axis=1)

In [14]:
total.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
import os
if not os.path.exists("data"):
    os.makedirs("data")
total.to_csv("data/income.csv", index=False)

object_cols = [col for col in total.columns if total[col].dtype == 'object']
print("Object columns:", object_cols)


Object columns: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country', 'income']


In [27]:
total=pd.read_csv("data/income.csv")
total['income'] = total['income'].str.strip().str.replace('.', '', regex=False)

total['income'].value_counts()

income
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [28]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def encode_data(df, log_file='encoding_log.txt'):
    df_copy = df.copy()
    with open(log_file, 'w', encoding='utf-8') as f:
        for col in df_copy.columns:
            if df_copy[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df_copy[col]):
                le = LabelEncoder()
                df_copy[col] = le.fit_transform(df_copy[col])
                
                # 输出每个类别和它的编码值
                mapping = {class_name: int(code) for code, class_name in enumerate(le.classes_)}
                f.write(f"Encoded column: {col}\n")
                for class_name, code in mapping.items():
                    f.write(f"  {class_name} -> {code}\n")
                f.write("\n")  # 换行隔开不同列的输出
    return df_copy

total_encoded = encode_data(total)
total_encoded.head()

/tmp/ipykernel_65662/1499531240.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df_copy[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df_copy[col]):
/tmp/ipykernel_65662/1499531240.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df_copy[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df_copy[col]):
/tmp/ipykernel_65662/1499531240.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if df_copy[col].dtype == 'object' or pd.api.types.is_categorical_dtype(df_copy[col]):
/tmp/ipykernel_65662/1499531240.py:8: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) inst

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,50,6,83311,9,13,2,4,0,4,1,0,0,13,39,0
2,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
3,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
4,28,4,338409,9,13,2,10,5,2,0,0,0,40,5,0


In [37]:
# 随机从total中抽取10000条数据
import random
random.seed(42)
total_labaled = total_encoded.sample(n=10000, random_state=42)
total_labaled.to_csv("data/total_labeled.csv", index=False)

total_unlabeled=total_encoded.drop(total_labaled.index).drop(columns=['income'])
total_unlabeled.to_csv("data/total_unlabeled.csv", index=False)

In [34]:
u=pd.read_csv("data/total_unlabeled.csv")
len(u)

38842

In [35]:
l=pd.read_csv("data/total_labeled.csv")
len(l)

10000

In [36]:
u.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,7,77516,9,13,4,1,1,4,1,2174,0,40,39,0
1,38,4,215646,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,234721,1,7,2,6,0,2,1,0,0,40,39,0
3,37,4,284582,12,14,2,4,5,4,0,0,0,40,39,0
4,31,4,45781,12,14,4,10,1,4,0,14084,0,50,39,1
